In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [26]:
import torch
import src.load as load
import numpy as np
from take_two.dataset import Data_Take2
import pandas as pd










In [2]:
folder_path = "../app/models"

In [11]:
models = {}

for dir in os.listdir(folder_path):
    sub_dir = os.path.join(folder_path, dir)
    models[dir] = {}
    for file in os.listdir(sub_dir):
        if file.endswith(".pt"):
            models[dir][file[:-3]] = torch.load(os.path.join(sub_dir, file))

        else:
            print('somthing went wrong')








In [12]:
for key in models.keys():
    print(models[key].keys())

dict_keys(['dist_18', 'yield_18', 'sched_18'])
dict_keys(['dist_20', 'yield_20', 'sched_20'])
dict_keys(['sched_16', 'dist_16', 'yield_16'])
dict_keys(['yield_11', 'dist_11', 'sched_11'])
dict_keys(['dist_10', 'yield_10', 'sched_10'])
dict_keys(['sched_17', 'yield_17', 'dist_17'])
dict_keys(['yield_19', 'dist_19', 'sched_19'])
dict_keys(['yield_2', 'sched_2', 'dist_2'])
dict_keys(['dist_5', 'sched_5', 'yield_5'])
dict_keys(['dist_4', 'sched_4', 'yield_4'])
dict_keys(['yield_3', 'sched_3', 'dist_3'])
dict_keys(['sched_12', 'dist_12', 'yield_12'])
dict_keys(['yield_15', 'dist_15', 'sched_15'])
dict_keys(['dist_14', 'yield_14', 'sched_14'])
dict_keys(['sched_13', 'yield_13', 'dist_13'])
dict_keys(['yield_6', 'sched_6', 'dist_6'])
dict_keys(['dist_1', 'sched_1', 'yield_1'])
dict_keys(['sched_8', 'yield_8', 'dist_8'])
dict_keys(['sched_9', 'yield_9', 'dist_9'])
dict_keys(['yield_7', 'sched_7', 'dist_7'])


In [18]:
meta, y, schedule, mappings, reverse_mappings = load.load_data('../data/processed/')

train_meta = meta[meta['Year'] != 2024]
test_meta = meta[meta['Year'] == 2024]

train_y = y[y.index.isin(train_meta.index)]
test_y = y[y.index.isin(test_meta.index)]

train_schedule = schedule[schedule.index.isin(train_meta.index)]
test_schedule = schedule[schedule.index.isin(test_meta.index)]

In [15]:
def make_dataset(meta, y, schedule, mappings):
    features = np.column_stack([
    meta['Ha'].to_numpy(),  # Hectares
    meta['WeekSin'].to_numpy(),  # Week sine
    meta['WeekCos'].to_numpy(),  # Week cosine
    meta['Year'].to_numpy() - 2010,                  # Year
    np.ones(len(meta))                    # Constant feature
])
    mapped_arrays = [meta[column].astype(str).map(mappings[column]).to_numpy() for column in ['ProducerCode','Parcel','Class','Type','Variety']]
    encoded_features = np.column_stack(mapped_arrays)

    climate_data = np.array(meta.ClimateSeries.to_list())
    schedule_data = schedule.values
    kilo_dist = (y.to_numpy() / y.to_numpy().sum(axis=1, keepdims=True)).cumsum(axis=1)
    yield_dist = np.log1p(y.to_numpy() / meta['Ha'].to_numpy()[:, np.newaxis])
    yield_log = np.log1p(y.to_numpy().sum(axis=1) / meta['Ha'].to_numpy())

    dataset = Data_Take2(
        features,
        encoded_features,
        climate_data,
        yield_dist,
        kilo_dist,
        yield_log,
        schedule_data
    )
    return dataset

In [19]:
train_dataset = make_dataset(train_meta, train_y, train_schedule, mappings)
test_dataset = make_dataset(test_meta, test_y, test_schedule, mappings)

In [22]:
train_meta['Kilos'] = train_y.sum(axis=1)

/var/folders/fb/78nymm551yn_7vm06j_3sk9w0000gn/T/ipykernel_88746/1319993333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_meta['Kilos'] = train_y.sum(axis=1)


In [24]:
train_schedule

,iqr_weeks,weighted_mean_weeks,first_harvest,end_harvest,uncertainty
0,4,18.170098,11,22,1.049875
1,2,16.101399,10,18,0.645584
2,3,15.972523,11,19,0.709523
3,4,16.546460,10,19,0.278180
4,1,14.495413,11,15,0.794334
...,...,...,...,...,...
3609,0,20.383747,17,20,1.219886
3610,0,18.000000,17,17,1.290552
3611,1,18.619048,17,18,1.168507
3612,0,19.000000,18,18,1.374906


In [29]:
historical_production = pd.concat([train_meta, train_schedule], axis = 1)

In [33]:
historical_types = historical_production.groupby('Type').agg({'Ha': 'sum', 'Kilos': 'sum'})

In [34]:
historical_types['Yield'] = historical_types.Kilos / historical_types.Ha


In [35]:
historical_types

,Ha,Kilos,Yield
Type,,,
Cherry Amarillo,7.2393,119180.0,16462.917685
Cherry Chocolate,2.8389,23947.0,8435.309451
Cherry Naranja,5.7427,106653.0,18571.926097
Cherry Rojo,194.0257,6262179.0,32274.997591
Cherry Rosa,5.6339,118570.0,21045.811960
Cien Dulce,0.8485,21065.0,24826.163819
Heirloom Cherry,95.7713,1359587.0,14196.184034
Pera,39.9433,716149.5,17929.152073
Pera Amarilla,30.5670,344105.0,11257.401773


In [30]:
production_plan = test_meta

In [31]:
production_plan

,TransplantDate,Parcel,Lot,ProducerCode,Variety,Ha,Class,Type,WeekTransplanted,WeekCos,WeekSin,Year,ClimateSeries
3614,2024-01-26,V02,13,VAP,King,0.8280,BSUF,Uva Roja,4,0.885456,0.464723,2024,"[[-0.3505129556, -0.7858435544, 0.0, 0.0, -0.6..."
3615,2024-01-26,V02,14,VAP,Prince,0.4416,BSUF,Uva Roja,4,0.885456,0.464723,2024,"[[-0.3505129556, -0.7858435544, 0.0, 0.0, -0.6..."
3616,2024-01-26,V02,14,VAP,Tymoty,0.4224,CHE,Cherry Rojo,4,0.885456,0.464723,2024,"[[-0.3505129556, -0.7858435544, 0.0, 0.0, -0.6..."
3617,2024-02-09,V02,17,VAP,King,0.4500,BSUF,Uva Roja,6,0.748511,0.663123,2024,"[[-1.1937728734, -1.1056878042, 0.300000000000..."
3618,2024-02-09,V02,17,VAP,Pera Amarilla Viv,0.0540,CHE,Pera Amarilla,6,0.748511,0.663123,2024,"[[-1.1937728734, -1.1056878042, 0.300000000000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3785,2024-07-25,S12,4,SGB,Top 2323,0.1530,CHE,Cherry Rosa,30,-0.885456,-0.464723,2024,"[[0.8452649894, 0.93700445, 0.0, 0.0, 0.485896..."
3786,2024-07-25,S12,4,SGB,Yoyomo,0.1020,BSUF,Uva Amarilla,30,-0.885456,-0.464723,2024,"[[0.8452649894, 0.93700445, 0.0, 0.0, 0.485896..."
3787,2024-07-29,S12,7,SGB,King,0.5100,BSUF,Uva Roja,31,-0.822984,-0.568065,2024,"[[0.3676142667, 0.3320816298, 0.0, 0.0, 0.5941..."
3788,2024-07-29,S12,8,SGB,King,0.0306,BSUF,Uva Roja,31,-0.822984,-0.568065,2024,"[[0.3676142667, 0.3320816298, 0.0, 0.0, 0.5941..."
